In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from itertools import count
import json
import argparse
import os
import random
import pandas as pd
import numpy as np
import pyBigWig

NARROWPEAK_SCHEMA = ["chr", "start", "end", "1", "2", "3", "4", "5", "6", "summit"]

peak_regions_df = pd.read_csv("/mnt/lab_data2/vir/tf_chr_atlas/temp/docker_modelling/ENCSR142IGM_peaks_inliers.bed.gz", sep='\t', names=NARROWPEAK_SCHEMA)
peak_regions_df['region']='peak'
peak_regions_df['ind']=range(len(peak_regions_df))
nonpeak_regions_df = pd.read_csv("/mnt/lab_data2/vir/tf_chr_atlas/temp/docker_modelling/ENCSR142IGM_gc_neg_only.bed.gz", sep='\t', names=NARROWPEAK_SCHEMA)
nonpeak_regions_df['region']='nonpeak'
nonpeak_regions_df['ind']=range(len(nonpeak_regions_df))


all_regions_df = pd.concat([peak_regions_df,nonpeak_regions_df])
all_regions_df['pos']=all_regions_df['start']+all_regions_df['summit']
all_regions_df.sort_values(by=['chr', 'pos'], inplace=True)
all_regions_df=all_regions_df.reset_index(drop=True)

print("Creating Splits")

group_dict = {}

inputlen=2114
max_jitter=32

cur_chrom = ''
cur_group = ''
last_pos = 0
for index,row in all_regions_df.iterrows():
    if cur_chrom != '':
        if row['chr'] != cur_chrom:
            cur_chrom = row['chr']
            cur_group += 1
            group_dict[cur_group] = [row]
        else:
            if row['pos'] <= int(last_pos) + int(inputlen) + int(2 * max_jitter):
                group_dict[cur_group].append(row)
            else:
                cur_group += 1
                group_dict[cur_group] = [row]
    else:
        cur_chrom = row['chr']
        cur_group = 0
        group_dict[cur_group] = [row]
    last_pos = row['pos']
    
groups = []
group_counts = []
bigwig = '/mnt/lab_data2/vir/tf_chr_atlas/temp/docker_modelling/ENCSR142IGM_plus.bigWig'
bw = pyBigWig.open(bigwig)

for group in group_dict:
    groups.append(group)
    sum = 0
    for element in group_dict[group]:
        labels = bw.values(element['chr'], int(element['pos'] - (inputlen // 2)), int(element['pos'] + (inputlen // 2)))
        labels = np.array(labels)
        labels = np.nan_to_num(labels)
        labels = np.sum(labels)
        sum += labels
    group_counts.append(sum)
group_df = pd.DataFrame({'groups': groups, 'group_counts': group_counts})
group_df.sort_values(by='group_counts', inplace=True)



Creating Splits


In [3]:
number_of_folds = 5

In [5]:
# split the data into x number of chuncks for x folds. allocate the chuncks to train, test, valid folds in unique ways
# across folds
chuncksets_dict={}
for fold in range(number_of_folds):
    chuncksets_dict[f"chuncksets_{fold}"]=list(range(fold,len(group_df),number_of_folds))
    
val_chuncks = list(range(0,number_of_folds))
print("val_chuncks:",val_chuncks)
test_chuncks = list(range(1,number_of_folds))+[0]
print("test_chuncks:",test_chuncks)

group_fold_df=pd.DataFrame(index=np.arange(len(group_df)))
for fold in range(number_of_folds):
    group_fold_df[f"fold{fold}"]='train'
    group_fold_df[f"fold{fold}"][chuncksets_dict[f"chuncksets_{val_chuncks[fold]}"]] = 'valid'
    group_fold_df[f"fold{fold}"][chuncksets_dict[f"chuncksets_{test_chuncks[fold]}"]] = 'test'
print(group_fold_df)

for fold in range(number_of_folds):
    print("fold:",fold)
    len(group_fold_df[group_fold_df[f"fold{fold}"]=="train"])/len(group_fold_df)

    len(group_fold_df[group_fold_df[f"fold{fold}"]=="test"])/len(group_fold_df)

    len(group_fold_df[group_fold_df[f"fold{fold}"]=="valid"])/len(group_fold_df)



val_chuncks: [0, 1, 2, 3, 4]
test_chuncks: [1, 2, 3, 4, 0]


In [8]:
# group_fold_dict = {}
# for fold in range(number_of_folds):
#     group_fold_dict[f"fold{fold}"]=[]

# count = 0
# valid_used = []



# for index,row in group_df.iterrows():
#     if index % 10000 == 0:
#         print(index)
#     if count % 2 == 0:
#         test_or_valid = 'valid'
#     else:
#         test_or_valid = 'test'
#     test_or_valid_fold = random.choice([i for i in range(number_of_folds) if i not in valid_used])
#     for fold in range(number_of_folds):
#         if fold != test_or_valid_fold:
#             group_fold_dict[f"fold{fold}"].append('train')
#         else:
#             group_fold_dict[f"fold{fold}"].append(test_or_valid)
#     count += 1
#     valid_used.append(test_or_valid_fold)
#     if len(valid_used) == number_of_folds:
#         valid_used = []


for fold in range(number_of_folds):
    group_df['fold' + str(fold)] = group_fold_df['fold' + str(fold)]
    
group_df
output_path ="."
print("Saving Splits")
for fold in range(number_of_folds):
    print("fold:",fold)
    for split in ['valid','train','test']:
        temp_lst = [group_dict.get(key) for key in group_df['groups'][group_df[f"fold{fold}"]==split]] 
        peak_indices = [i['ind'] for b in map(lambda x:[x] if not isinstance(x, list) else x, temp_lst) for i in b if i['region']=='peak']
        nonpeak_indices = [i['ind'] for b in map(lambda x:[x] if not isinstance(x, list) else x, temp_lst) for i in b if i['region']=='nonpeak']
        print("split:",split)
        print("proportion of peaks:",len(peak_indices)/len(peak_regions_df))
        print("length of nonpeaks:",len(nonpeak_indices)/len(nonpeak_regions_df))
        
        # f = open(f"{output_path}/loci_{split}_indices_fold{fold}.txt", "w")
        # for items in peak_indices:
        #     f.writelines(str(items)+'\n')
        # f.close()
        # f = open(f"{output_path}/background_{split}_indices_fold{fold}.txt", "w")
        # for items in nonpeak_indices:
        #     f.writelines(str(items)+'\n')
        # f.close()
        # nonpeak_regions_df.iloc[nonpeak_indices,0:10].to_csv(f"{output_path}/background_peaks_{split}_fold{fold}.bed",sep="\t",header=False,index=False)
        # peak_regions_df.iloc[peak_indices,0:10].to_csv(f"{output_path}/peaks_{split}_fold{fold}.bed",sep="\t",header=False,index=False)
    print("\n")

from plotnine import *
group_df["log_groupcounts"]=np.log10(group_df["group_counts"]+1)
for fold in range(number_of_folds):
    print("fold:",fold)
    plot = (ggplot(group_df,aes("log_groupcounts"))
                    +facet_wrap(f"fold{fold}")
                    +geom_histogram(bins=30)
                    +theme_classic()
           )
    plot.save(f'fold{fold}_counts_histogram_plot.png')

,groups,group_counts,fold0,fold1,fold2,fold3,fold4
164291,164291,0.0,test,valid,train,train,train
127077,127077,0.0,train,test,valid,train,train
150211,150211,0.0,test,valid,train,train,train
205607,205607,0.0,train,test,valid,train,train
205606,205606,0.0,test,valid,train,train,train
...,...,...,...,...,...,...,...
45877,45877,5272.0,train,test,valid,train,train
101000,101000,5402.0,valid,train,train,train,test
46663,46663,5468.0,train,train,test,valid,train
76680,76680,7740.0,valid,train,train,train,test


Saving Splits
fold: 0
split: valid
proportion of peaks: 0.2015091219184454
length of nonpeaks: 0.20041085353439977
split: train
proportion of peaks: 0.5962582045583751
length of nonpeaks: 0.6002745164871102
split: test
proportion of peaks: 0.2022326735231795
length of nonpeaks: 0.19931462997849006


fold: 1
split: valid
proportion of peaks: 0.2022326735231795
length of nonpeaks: 0.19931462997849006
split: train
proportion of peaks: 0.5983254948576153
length of nonpeaks: 0.6004725736841863
split: test
proportion of peaks: 0.19944183161920512
length of nonpeaks: 0.20021279633732364


fold: 2
split: valid
proportion of peaks: 0.19944183161920512
length of nonpeaks: 0.20021279633732364
split: train
proportion of peaks: 0.6007028787017417
length of nonpeaks: 0.5999474918128682
split: test
proportion of peaks: 0.1998552896790532
length of nonpeaks: 0.19983971184980817


fold: 3
split: valid
proportion of peaks: 0.1998552896790532
length of nonpeaks: 0.19983971184980817
split: train
proportio

In [16]:
from plotnine import *
group_df["log_groupcounts"]=np.log10(group_df["group_counts"]+1)
for fold in range(number_of_folds):
    print("fold:",fold)
    plot = (ggplot(group_df,aes("log_groupcounts"))
                    +facet_wrap(f"fold{fold}")
                    +geom_histogram(bins=30)
                    +theme_classic()
           )
    plot.save(f'fold{fold}_counts_histogram_plot.png')

fold: 0


/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:721: PlotnineWarning: Saving 6.4 x 4.8 in image.
/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: fold0_counts_histogram_plot.jpg


fold: 1


/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:721: PlotnineWarning: Saving 6.4 x 4.8 in image.
/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: fold1_counts_histogram_plot.jpg


fold: 2


/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:721: PlotnineWarning: Saving 6.4 x 4.8 in image.
/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: fold2_counts_histogram_plot.jpg


fold: 3


/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:721: PlotnineWarning: Saving 6.4 x 4.8 in image.
/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: fold3_counts_histogram_plot.jpg


fold: 4


/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:721: PlotnineWarning: Saving 6.4 x 4.8 in image.
/users/vir/anaconda3/envs/basepairmodels_latest/lib/python3.7/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: fold4_counts_histogram_plot.jpg


In [23]:
%%bash
bedtools intersect -a /mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_valid_fold0.bed -b /mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_valid_fold0.bed


In [25]:

%%bash
bedtools intersect -a /mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_test_fold0.bed -b /mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_valid_fold0.bed



In [81]:
%%bash
for fold in {0..4}
do
echo "fold${fold}"
bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_test_fold${fold}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_train_fold${fold}.bed"
bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_test_fold${fold}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_train_fold${fold}.bed"
bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_test_fold${fold}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_valid_fold${fold}.bed"
bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_test_fold${fold}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_valid_fold${fold}.bed"
bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_train_fold${fold}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_valid_fold${fold}.bed"
bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_train_fold${fold}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_valid_fold${fold}.bed"
done


fold0
fold1
fold2
fold3
fold4


In [73]:
%%bash
for ((foldA = 0,foldB = 1 ; foldA <= 4,foldB <= 5 ; foldA++, foldB++))
do
    if [[ "${foldB}" == "5" ]]; then
        echo "${foldA},0"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_train_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_train_fold0.bed"
    else
        echo "${foldA},${foldB}"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_train_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_train_fold${foldB}.bed"
    fi
done

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [78]:
%%bash
for ((foldA = 0,foldB = 1 ; foldA <= 4,foldB <= 5 ; foldA++, foldB++))
do
    if [[ "${foldB}" == "5" ]]; then
        echo "${foldA},0"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_train_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_train_fold0.bed"
    else
        echo "${foldA},${foldB}"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_train_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_train_fold${foldB}.bed"
    fi
done

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [74]:
%%bash
for ((foldA = 0,foldB = 1 ; foldA <= 4,foldB <= 5 ; foldA++, foldB++))
do
    if [[ "${foldB}" == "5" ]]; then
        echo "${foldA},0"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_valid_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_valid_fold0.bed"
    else
        echo "${foldA},${foldB}"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_valid_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_valid_fold${foldB}.bed"
    fi
done

0,1
1,2
2,3
3,4
4,0


In [75]:
%%bash
for ((foldA = 0,foldB = 1 ; foldA <= 4,foldB <= 5 ; foldA++, foldB++))
do
    if [[ "${foldB}" == "5" ]]; then
        echo "${foldA},0"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_valid_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_valid_fold0.bed"
    else
        echo "${foldA},${foldB}"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_valid_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_valid_fold${foldB}.bed"
    fi
done

0,1
1,2
2,3
3,4
4,0


In [76]:
%%bash
for ((foldA = 0,foldB = 1 ; foldA <= 4,foldB <= 5 ; foldA++, foldB++))
do
    if [[ "${foldB}" == "5" ]]; then
        echo "${foldA},0"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_test_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_test_fold0.bed"
    else
        echo "${foldA},${foldB}"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_test_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/background_peaks_test_fold${foldB}.bed"
    fi
done

0,1
1,2
2,3
3,4
4,0


In [77]:
%%bash
for ((foldA = 0,foldB = 1 ; foldA <= 4,foldB <= 5 ; foldA++, foldB++))
do
    if [[ "${foldB}" == "5" ]]; then
        echo "${foldA},0"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_test_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_test_fold0.bed"
    else
        echo "${foldA},${foldB}"
        bedtools intersect -a "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_test_fold${foldA}.bed" -b "/mnt/lab_data2/vir/tf_chr_atlas/scripts/tf-atlas-pipeline/anvil/modeling/peaks_test_fold${foldB}.bed"
    fi
done

0,1
1,2
2,3
3,4
4,0
